# Data Wrangling

## Data Cleansing and Organization

#### Imports

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy import stats

In [2]:
df = pd.read_csv('waterQuality1.csv')

In [3]:
df.head()

,aluminium,ammonia,arsenic,barium,cadmium,chloramine,chromium,copper,flouride,bacteria,...,lead,nitrates,nitrites,mercury,perchlorate,radium,selenium,silver,uranium,is_safe
0,1.65,9.08,0.04,2.85,0.007,0.35,0.83,0.17,0.05,0.20,...,0.054,16.08,1.13,0.007,37.75,6.78,0.08,0.34,0.02,1
1,2.32,21.16,0.01,3.31,0.002,5.28,0.68,0.66,0.90,0.65,...,0.100,2.01,1.93,0.003,32.26,3.21,0.08,0.27,0.05,1
2,1.01,14.02,0.04,0.58,0.008,4.24,0.53,0.02,0.99,0.05,...,0.078,14.16,1.11,0.006,50.28,7.07,0.07,0.44,0.01,0
3,1.36,11.33,0.04,2.96,0.001,7.23,0.03,1.66,1.08,0.71,...,0.016,1.41,1.29,0.004,9.12,1.72,0.02,0.45,0.05,1
4,0.92,24.33,0.03,0.20,0.006,2.67,0.69,0.57,0.61,0.13,...,0.117,6.74,1.11,0.003,16.90,2.41,0.02,0.06,0.02,1


#### All columns are relevant to scope of project - no column drop is needed
#### Search for and remove any duplicates in data

In [4]:
df.duplicated().value_counts()

False    7999
dtype: int64

#### Check data types

In [5]:
df.dtypes

aluminium      float64
ammonia         object
arsenic        float64
barium         float64
cadmium        float64
chloramine     float64
chromium       float64
copper         float64
flouride       float64
bacteria       float64
viruses        float64
lead           float64
nitrates       float64
nitrites       float64
mercury        float64
perchlorate    float64
radium         float64
selenium       float64
silver         float64
uranium        float64
is_safe         object
dtype: object

#### Convert ammonia and is_safe columns to numeric data

In [6]:
df['ammonia'] = pd.to_numeric(df['ammonia'], errors='coerce')
df['is_safe'] = pd.to_numeric(df['is_safe'], errors='coerce')

#### Check data types again

In [7]:
df.dtypes

aluminium      float64
ammonia        float64
arsenic        float64
barium         float64
cadmium        float64
chloramine     float64
chromium       float64
copper         float64
flouride       float64
bacteria       float64
viruses        float64
lead           float64
nitrates       float64
nitrites       float64
mercury        float64
perchlorate    float64
radium         float64
selenium       float64
silver         float64
uranium        float64
is_safe        float64
dtype: object

#### Check for missing data

In [8]:
# Check entire dataframe for missing values
df.isna().values.any()

True

In [9]:
# Check which columns are missing values
df.isna().any()

aluminium      False
ammonia         True
arsenic        False
barium         False
cadmium        False
chloramine     False
chromium       False
copper         False
flouride       False
bacteria       False
viruses        False
lead           False
nitrates       False
nitrites       False
mercury        False
perchlorate    False
radium         False
selenium       False
silver         False
uranium        False
is_safe         True
dtype: bool

In [10]:
# Check the total number of missing values in each column
df.isna().sum()

aluminium      0
ammonia        3
arsenic        0
barium         0
cadmium        0
chloramine     0
chromium       0
copper         0
flouride       0
bacteria       0
viruses        0
lead           0
nitrates       0
nitrites       0
mercury        0
perchlorate    0
radium         0
selenium       0
silver         0
uranium        0
is_safe        3
dtype: int64

In [11]:
# Check how many total rows are missing values
df.isna().any(axis=1).sum()

3

#### 3 rows with missing values is a very small percentage of the dataset
#### Can handle missing data by deleting the 3 rows

In [12]:
df.dropna(inplace=True);

In [13]:
df.isna().values.any()

False

#### With no missing data, convert is_safe column to integers

In [14]:
df['is_safe'] = df['is_safe'].astype(int)

In [15]:
df.dtypes

aluminium      float64
ammonia        float64
arsenic        float64
barium         float64
cadmium        float64
chloramine     float64
chromium       float64
copper         float64
flouride       float64
bacteria       float64
viruses        float64
lead           float64
nitrates       float64
nitrites       float64
mercury        float64
perchlorate    float64
radium         float64
selenium       float64
silver         float64
uranium        float64
is_safe          int32
dtype: object

#### Correct spelling and accuracy of column names

In [16]:
df.rename({'aluminium': 'aluminum',
            'flouride': 'fluoride',
            'is_safe': 'potability'}, 
            axis=1, inplace=True)

df.columns

Index(['aluminum', 'ammonia', 'arsenic', 'barium', 'cadmium', 'chloramine',
       'chromium', 'copper', 'fluoride', 'bacteria', 'viruses', 'lead',
       'nitrates', 'nitrites', 'mercury', 'perchlorate', 'radium', 'selenium',
       'silver', 'uranium', 'potability'],
      dtype='object')

#### Write data to csv

In [17]:
df.to_csv('./clean_df.csv', index=False)

## Feature Engineering
#### Goal: Make explanatory variables (the chemical and biological characteristics) better suited to the outcome variable (potability). Check conditions for logistic regression

#### Assumption 1 of Logistic Regression: Appropriate Outcome Type

In [18]:
# Outcome variable = potability
# potability should only have two unique outcomes

df['potability'].nunique()

2

#### Assumption 1 is met as potability is binary
#### Assumption 2: Linearity of independent variables and log-odds

In [19]:
def check_linearity(dframe):
    """ Function that makes use of the Box-Tidwell test to check for linearity
        between predictors and the logit
        
        params: dframe - dataframe to analyze
        
        output: prints significance levels of analyzed relationships."""

    
    # Define continuous variables
    cont_vars = dframe.iloc[:, 0:-1]    

    # Apply natural log to continuous variables
    for var in cont_vars:
        dframe[f'{var}:Log_{var}'] = dframe[var].apply(lambda x: x * np.log(x))

    dframe.fillna(0.00, inplace=True)

    # Get column list of continuous variables
    dframe.drop(['potability'], axis=1, inplace=True)
    keep_cols = dframe.columns.to_list()

    # Create new dataframe from continuous variables and interaction terms
    lin_df = dframe[keep_cols]

    # Add a constant term to df
    lin_df_const = sm.add_constant(lin_df)

    # Build the model to fit data
    logit_results = sm.GLM(list(df['potability']), lin_df_const, family=sm.families.Binomial()).fit()

    # Send results summary
    return logit_results.summary()

In [20]:
check_df = pd.read_csv('clean_df.csv')

In [21]:
results = check_linearity(check_df)
results

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                 7996
Model:                            GLM   Df Residuals:                     7955
Model Family:                Binomial   Df Model:                           40
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1563.4
Date:                Sun, 30 Jan 2022   Deviance:                       3126.9
Time:                        21:37:26   Pearson chi2:                 1.28e+04
No. Iterations:                     7   Pseudo R-squ. (CS):             0.2730
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const                           0.9572      0.574      1.667      0.095      -0.168       2.082
aluminum                        3.3109      0.163     20.356      0.000       2.992       3.630
ammonia                         0.0541      0.053      1.019      0.308      -0.050       0.158
arsenic                        -2.9783      0.321     -9.287      0.000      -3.607      -2.350
barium                          0.0030      0.182      0.016      0.987      -0.354       0.360
cadmium                        34.7279      9.159      3.792      0.000      16.776      52.679
chloramine                      1.1895      0.144      8.234      0.000       0.906       1.473
chromium                        0.2947      0.208      1.415      0.157      -0.114       0.703
copper                         -1.0001      0.181     -5.514      0.000      -1.356      -0.645
fluoride                        0.2751      0.224      1.227      0.220      -0.164       0.714
bacteria                        2.5345      0.402      6.307      0.000       1.747       3.322
viruses                        -3.1873      0.440     -7.249      0.000      -4.049      -2.326
lead                           -2.9502      3.742     -0.788      0.430     -10.285       4.384
nitrates                       -0.1138      0.080     -1.426      0.154      -0.270       0.043
nitrites                       -0.4168      0.332     -1.255      0.209      -1.068       0.234
mercury                       128.6711    192.801      0.667      0.505    -249.211     506.554
perchlorate                     0.3479      0.036      9.775      0.000       0.278       0.418
radium                         -0.0119      0.127     -0.094      0.925      -0.261       0.237
selenium                       -4.3464      9.504     -0.457      0.647     -22.974      14.282
silver                         -0.0011      0.675     -0.002      0.999      -1.324       1.322
uranium                       -22.0120     10.589     -2.079      0.038     -42.767      -1.257
aluminum:Log_aluminum          -1.7042      0.100    -16.961      0.000      -1.901      -1.507
ammonia:Log_ammonia            -0.0243      0.015     -1.610      0.107      -0.054       0.005
arsenic:Log_arsenic             3.1630      0.636      4.972      0.000       1.916       4.410
barium:Log_barium               0.0141      0.113      0.125      0.901      -0.207       0.235
cadmium:Log_cadmium            22.3757      4.147      5.395      0.000      14.247      30.504
chloramine:Log_chloramine      -0.4931      0.064     -7.648      0.000      -0.619      -0.367
chromium:Log_chromium          -2.3494      0.570     -4.119      0.000      -3.467      -1.231
copper:Log_copper               0.5655      0.219      2.585      0.010       0.137       0.994
fluoride:Lo

#### Ten predictors have nonlinearity with the logit (the Log columns where p < 0.05)

#### Solution attempt: Transform data into different order to capture non-linearity

In [22]:
check_df = pd.read_csv('clean_df.csv')

check_df.iloc[:, [9, 10]] = \
            check_df.iloc[:, [9, 10]] \
            .apply(lambda x: x ** 3)

check_df.iloc[:, [15]] = \
            check_df.iloc[:, [15]] \
            .apply(lambda x: x ** 5)

check_df.iloc[:, [2, 4, 7]] = \
            check_df.iloc[:, [2, 4, 7]] \
            .apply(lambda x: x ** 0.5)

check_df.iloc[:, [0, 5]] = \
            check_df.iloc[:, [0, 5]] \
            .apply(lambda x: x ** 0.20)

check_df.iloc[:, [6, 18]] = \
            check_df.iloc[:, [6, 18]] \
            .apply(lambda x: x ** 0.333)

In [23]:
results = check_linearity(check_df)
results

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                 7996
Model:                            GLM   Df Residuals:                     7958
Model Family:                Binomial   Df Model:                           37
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1650.1
Date:                Sun, 30 Jan 2022   Deviance:                       3300.2
Time:                        21:37:26   Pearson chi2:                 1.67e+04
No. Iterations:                    11   Pseudo R-squ. (CS):             0.2570
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const                          -4.4083      1.335     -3.302      0.001      -7.025      -1.792
aluminum                        6.3611      1.028      6.186      0.000       4.346       8.377
ammonia                         0.0631      0.051      1.226      0.220      -0.038       0.164
arsenic                        -3.4265      0.294    -11.662      0.000      -4.002      -2.851
barium                          0.0860      0.176      0.488      0.626      -0.260       0.431
cadmium                        -7.3815      1.574     -4.689      0.000     -10.467      -4.296
chloramine                      2.8841      0.790      3.651      0.000       1.336       4.432
chromium                        0.8493      0.347      2.445      0.014       0.168       1.530
copper                         -1.1523      0.286     -4.026      0.000      -1.713      -0.591
fluoride                        0.3042      0.216      1.406      0.160      -0.120       0.728
bacteria                        2.6091      0.671      3.887      0.000       1.293       3.925
viruses                        -2.7344      0.684     -3.999      0.000      -4.075      -1.394
lead                           -2.6564      3.619     -0.734      0.463      -9.750       4.437
nitrates                       -0.0548      0.078     -0.703      0.482      -0.208       0.098
nitrites                       -0.2704      0.325     -0.831      0.406      -0.908       0.367
mercury                       159.0951    185.819      0.856      0.392    -205.104     523.294
perchlorate                 -1.802e-08    1.8e-08     -1.002      0.316   -5.33e-08    1.72e-08
radium                          0.0409      0.122      0.334      0.738      -0.199       0.281
selenium                       -4.0756      9.222     -0.442      0.659     -22.150      13.998
silver                         -2.4096      0.313     -7.688      0.000      -3.024      -1.795
uranium                       -20.0622     10.283     -1.951      0.051     -40.216       0.091
aluminum:Log_aluminum          -1.7239      1.135     -1.518      0.129      -3.949       0.501
ammonia:Log_ammonia            -0.0269      0.015     -1.843      0.065      -0.056       0.002
arsenic:Log_arsenic            -0.1325      0.563     -0.235      0.814      -1.236       0.971
barium:Log_barium              -0.0217      0.110     -0.197      0.844      -0.237       0.194
cadmium:Log_cadmium            -1.5207      1.459     -1.042      0.297      -4.381       1.339
chloramine:Log_chloramine      -0.9877      0.773     -1.277      0.202      -2.503       0.528
chromium:Log_chromium           0.1770      0.622      0.285      0.776      -1.042       1.396
copper:Log_copper               0.3184      0.410      0.776      0.438      -0.486       1.123
fluoride:Lo

#### All non-linearity is now captured and assumption 2 is met

#### Assumption 3: No strongly influential outliers

In [24]:
def get_outliers(dframe):
    """Function to generate z-scores for each data point and
        determine whether points are outliers
    
        params: dframe - dataframe to check for outliers
    
        output: array of outliers in dframe"""
    
    
    z = np.abs(stats.zscore(check_df.iloc[:, 0:20]))
    threshold = 3.0
    return np.where(z > threshold)[0]

In [25]:
# Find outliers and drop the rows from dataframe

outliers = get_outliers(check_df)
check_df.drop(outliers, inplace=True)

In [26]:
# Get shape of dataframe
check_df.shape

(7519, 40)

#### After dropping 480 rows with outliers, assumption 3 is met

#### Assumption 4: Absence of Multicollinearity

In [27]:
def calc_vif(dframe):
    """ Function to measure degree of multicollinearity using the
        variance inflation factor (VIF), where a VIF above 5 indicates
        a high degree of multicollinearity
        
        params: dframe - the dataframe to check for multicollinearity
        
        output: vif - the variance inflation factor of all columns"""
    
    
    vif = pd.DataFrame()
    vif["variables"] = dframe.columns
    
    vif["VIF"] = [variance_inflation_factor(dframe.values, i) \
                  for i in range(dframe.shape[1])]
    return vif

In [28]:
calc_vif(check_df.iloc[:, 0:20])

,variables,VIF
0,aluminum,8.687169
1,ammonia,3.604091
2,arsenic,4.267025
3,barium,3.903454
4,cadmium,5.242783
5,chloramine,18.898124
6,chromium,11.642773
7,copper,4.742568
8,fluoride,3.922369
9,bacteria,3.333710


In [29]:
# Silver metals like aluminum, cadmium, chromium, and silver seem to be interdependent
# Nitrites are likely dependent upon nitrates
# Chloramine is related to ammonia and fluoride

# Drop these columns from the dataset then check vif values again

check_df.drop(['aluminum', 
               'chloramine', 
               'chromium', 
               'nitrites', 
               'silver'], axis=1, inplace=True)

calc_vif(check_df.iloc[:, 0:15])

,variables,VIF
0,ammonia,3.412974
1,arsenic,3.812450
2,barium,3.308324
3,cadmium,4.991473
4,copper,4.170518
5,fluoride,3.792120
6,bacteria,2.999732
7,viruses,3.194146
8,lead,3.745129
9,nitrates,3.783070


#### After dropping 5 columns that related strongly to other attributes, assumption 4 is met

#### Assumption 5: Independence of observations

In [30]:
# Independence assumption is automatically met because the data is drawn 
# from separate bodies of water, with each row representing 
# a different water resource

#### Because each row of data is a separate body of water, assumption 5 is met

#### Assumption 6: Sufficiently large sample size

In [31]:
# Rule of thumb: there should be 10 observations with the least frequent outcome
# However, given the high resolution of scientific data, this is improbable
# Another rule of thumb: number of observations should be > 500

len(check_df)

7519

#### With a total of 7519 observations, the 6th and final assumption for logistic regression has been met

#### Write clean, organized, engineered dataset to csv file for future use

In [32]:
check_df.to_csv('./ready_df.csv', index=False)